In [ ]:
import numpy as np 
import tensorflow as tf
import pandas as pd
from tqdm import tqdm
import os
from cv2 import imread, createCLAHE 
import cv2
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
from numpy import savez_compressed
import matplotlib.pyplot as plt 
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from os import listdir
from numpy import asarray

from pydicom import dcmread

import glob 
from os import listdir
from numpy import asarray
from numpy import vstack

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from numpy import savez_compressed
import numpy as np 



In [ ]:
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from matplotlib import pyplot

In [ ]:
def plotMask(X,y):
    sample = []
    
    for i in range(6):
        left = X[i]
        right = y[i]
        combined = np.hstack((left,right))
        sample.append(combined)
        
        
    for i in range(0,6,3):

        plt.figure(figsize=(25,10))
        
        plt.subplot(2,3,1+i)
        plt.imshow(sample[i].astype('uint8'))
        
        plt.subplot(2,3,2+i)
        plt.imshow(sample[i+1].astype('uint8'))
        
        
        plt.subplot(2,3,3+i)
        plt.imshow(sample[i+2].astype('uint8'))
        
        plt.show()

# plot training and testing images 

In [ ]:
# Load training and testing data
datatrain = load("../input/unetdata/training_256.npz")
X_train, y_train = datatrain['arr_0'], datatrain['arr_1']
plotMask(X_train, y_train)

In [ ]:
datatest = load("../input/unetdata/testing_256.npz")
X_test, y_test = datatest['arr_0'], datatest['arr_1']
plotMask(X_test, y_test)

In [ ]:
X_test.shape

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
assert X_train.shape == y_train.shape
assert X_test.shape == y_test.shape

In [ ]:

assert X_train.shape == y_train.shape
assert X_test.shape == y_test.shape
images = np.concatenate((X_train,X_test),axis=0)
mask  = np.concatenate((y_train,y_test),axis=0)

# model

In [ ]:
def soft_dice_loss(y_true, y_pred, axis=(1, 2, 3), 
                   epsilon=0.00001):
    """
    Compute mean soft dice loss over all abnormality classes.

    Args:
        y_true (Tensorflow tensor): tensor of ground truth values for all classes.
                                    shape: (num_classes, x_dim, y_dim, z_dim)
        y_pred (Tensorflow tensor): tensor of soft predictions for all classes.
                                    shape: (num_classes, x_dim, y_dim, z_dim)
        axis (tuple): spatial axes to sum over when computing numerator and
                      denominator in formula for dice loss.
                      Hint: pass this as the 'axis' argument to the K.sum
                            and K.mean functions.
        epsilon (float): small constant added to numerator and denominator to
                        avoid divide by 0 errors.
    Returns:
        dice_loss (float): computed value of dice loss.     
    """

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    dice_numerator = 2. * K.sum(y_true * y_pred, axis=axis) + epsilon
    dice_denominator = K.sum(y_true**2, axis=axis) + K.sum(y_pred**2, axis=axis) + epsilon
    dice_loss = 1- K.mean((dice_numerator)/(dice_denominator))

    ### END CODE HERE ###

    return dice_loss

# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def dice_coefficient(y_true, y_pred, axis=(1, 2, 3), 
                     epsilon=0.00001):
    """
    Compute mean dice coefficient over all abnormality classes.

    Args:
        y_true (Tensorflow tensor): tensor of ground truth values for all classes.
                                    shape: (num_classes, x_dim, y_dim, z_dim)
        y_pred (Tensorflow tensor): tensor of predictions for all classes.
                                    shape: (num_classes, x_dim, y_dim, z_dim)
        axis (tuple): spatial axes to sum over when computing numerator and
                      denominator of dice coefficient.
                      Hint: pass this as the 'axis' argument to the K.sum
                            and K.mean functions.
        epsilon (float): small constant add to numerator and denominator to
                        avoid divide by 0 errors.
    Returns:
        dice_coefficient (float): computed value of dice coefficient.     
    """

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    dice_numerator = 2. * K.sum(y_true * y_pred, axis=axis) + epsilon
    dice_denominator = K.sum(y_true**2, axis=axis) + K.sum(y_pred**2, axis=axis) + epsilon
    
    dice_coefficient = K.mean((dice_numerator)/(dice_denominator))
    
    ### END CODE HERE ###

    return dice_coefficient
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K

def gating_signal(input, out_size, batch_norm=False):
    """
    resize the down layer feature map into the same dimension as the up layer feature map
    using 1x1 conv
    :return: the gating feature map with the same dimension of the up layer feature map
    """
    x = layers.Conv2D(out_size, (1, 1), padding='same')(input)
    if batch_norm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

def attention_block(x, gating, inter_shape):
    shape_x = K.int_shape(x)
    shape_g = K.int_shape(gating)

# Getting the x signal to the same shape as the gating signal
    theta_x = layers.Conv2D(inter_shape, (2, 2), strides=(2, 2), padding='same')(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

# Getting the gating signal to the same number of filters as the inter_shape
    phi_g = layers.Conv2D(inter_shape, (1, 1), padding='same')(gating)
    upsample_g = layers.Conv2DTranspose(inter_shape, (3, 3),
                                 strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]),
                                 padding='same')(phi_g)  # 16

    concat_xg = layers.add([upsample_g, theta_x])
    act_xg = layers.Activation('relu')(concat_xg)
    psi = layers.Conv2D(1, (1, 1), padding='same')(act_xg)
    sigmoid_xg = layers.Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = layers.UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)  # 32

    upsample_psi = repeat_elem(upsample_psi, shape_x[3])

    y = layers.multiply([upsample_psi, x])

    result = layers.Conv2D(shape_x[3], (1, 1), padding='same')(y)
    result_bn = layers.BatchNormalization()(result)
    return result_bn


def repeat_elem(tensor, rep):
    # lambda function to repeat Repeats the elements of a tensor along an axis
    #by a factor of rep.
    # If tensor has shape (None, 256,256,3), lambda will return a tensor of shape 
    #(None, 256,256,6), if specified axis=3 and rep=2.

     return layers.Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3),
                          arguments={'repnum': rep})(tensor)


def res_conv_block(x, filter_size, size, dropout, batch_norm=False):
    '''
    Residual convolutional layer.
    Two variants....
    Either put activation function before the addition with shortcut
    or after the addition (which would be as proposed in the original resNet).
    
    1. conv - BN - Activation - conv - BN - Activation 
                                          - shortcut  - BN - shortcut+BN
                                          
    2. conv - BN - Activation - conv - BN   
                                     - shortcut  - BN - shortcut+BN - Activation                                     
    
    Check fig 4 in https://arxiv.org/ftp/arxiv/papers/1802/1802.06955.pdf
    '''

    conv = layers.Conv2D(size, (filter_size, filter_size), padding='same')(x)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation('relu')(conv)
    
    conv = layers.Conv2D(size, (filter_size, filter_size), padding='same')(conv)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    #conv = layers.Activation('relu')(conv)    #Activation before addition with shortcut
    if dropout > 0:
        conv = layers.Dropout(dropout)(conv)

    shortcut = layers.Conv2D(size, kernel_size=(1, 1), padding='same')(x)
    if batch_norm is True:
        shortcut = layers.BatchNormalization(axis=3)(shortcut)

    res_path = layers.add([shortcut, conv])
    res_path = layers.Activation('relu')(res_path)    #Activation after addition with shortcut (Original residual block)
    return res_path

def Attention_ResUNet(input_shape, NUM_CLASSES=3, dropout_rate=0.0, batch_norm=False):
    '''
    Rsidual UNet, with attention 
    
    '''
    # network structure
    FILTER_NUM = 64 # number of basic filters for the first layer
    FILTER_SIZE = 3 # size of the convolutional filter
    UP_SAMP_SIZE = 2 # size of upsampling filters
    # input data
    # dimension of the image depth
    inputs = layers.Input(input_shape, dtype=tf.float32)
    axis = 3

    # Downsampling layers
    # DownRes 1, double residual convolution + pooling
    conv_128 = res_conv_block(inputs, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)
    pool_64 = layers.MaxPooling2D(pool_size=(2,2))(conv_128)
    # DownRes 2
    conv_64 = res_conv_block(pool_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    pool_32 = layers.MaxPooling2D(pool_size=(2,2))(conv_64)
    # DownRes 3
    conv_32 = res_conv_block(pool_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    pool_16 = layers.MaxPooling2D(pool_size=(2,2))(conv_32)
    # DownRes 4
    conv_16 = res_conv_block(pool_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    pool_8 = layers.MaxPooling2D(pool_size=(2,2))(conv_16)
    # DownRes 5, convolution only
    conv_8 = res_conv_block(pool_8, FILTER_SIZE, 16*FILTER_NUM, dropout_rate, batch_norm)

    # Upsampling layers
    # UpRes 6, attention gated concatenation + upsampling + double residual convolution
    gating_16 = gating_signal(conv_8, 8*FILTER_NUM, batch_norm)
    att_16 = attention_block(conv_16, gating_16, 8*FILTER_NUM)
    up_16 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(conv_8)
    up_16 = layers.concatenate([up_16, att_16], axis=axis)
    up_conv_16 = res_conv_block(up_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 7
    gating_32 = gating_signal(up_conv_16, 4*FILTER_NUM, batch_norm)
    att_32 = attention_block(conv_32, gating_32, 4*FILTER_NUM)
    up_32 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_16)
    up_32 = layers.concatenate([up_32, att_32], axis=axis)
    up_conv_32 = res_conv_block(up_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 8
    gating_64 = gating_signal(up_conv_32, 2*FILTER_NUM, batch_norm)
    att_64 = attention_block(conv_64, gating_64, 2*FILTER_NUM)
    up_64 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_32)
    up_64 = layers.concatenate([up_64, att_64], axis=axis)
    up_conv_64 = res_conv_block(up_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 9
    gating_128 = gating_signal(up_conv_64, FILTER_NUM, batch_norm)
    att_128 = attention_block(conv_128, gating_128, FILTER_NUM)
    up_128 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_64)
    up_128 = layers.concatenate([up_128, att_128], axis=axis)
    up_conv_128 = res_conv_block(up_128, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)

    # 1*1 convolutional layers
    
    conv_final = layers.Conv2D(NUM_CLASSES, kernel_size=(1,1))(up_conv_128)
    conv_final = layers.BatchNormalization(axis=axis)(conv_final)
    conv_final = layers.Activation('sigmoid')(conv_final)  #Change to softmax for multichannel

    # Model integration
    model = models.Model(inputs, conv_final, name="AttentionResUNet")
    return model
model = Attention_ResUNet(input_shape=(256,256,3))
model.compile(optimizer=Adam(lr=0.001), loss=soft_dice_loss,metrics=[dice_coefficient, 'binary_accuracy'])
model.summary()

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = keras.flatten(y_true)
    y_pred_f = keras.flatten(y_pred)
    intersection = keras.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('cxr_reg')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, 
                                   patience=5,
                                   verbose=1, mode='min', epsilon=0.0002, cooldown=2, min_lr=1e-7)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=50) # probably needs to be more patient, but kaggle time is limited
callbacks_list = [checkpoint, early, reduceLROnPlat]

In [ ]:
from IPython.display import clear_output
from tensorflow.keras.optimizers import Adam 
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
model.compile(optimizer=Adam(lr=0.001), 
              loss=[soft_dice_loss], 
           metrics = [dice_coefficient])

train_vol, validation_vol, train_seg, validation_seg = train_test_split((images)/255, 
                                                            (mask>127).astype(np.float32), 
                                                            test_size = 0.1,random_state = 2018)

train_vol, test_vol, train_seg, test_seg = train_test_split(train_vol,train_seg, 
                                                            test_size = 0.1, 
                                                            random_state = 2018)



In [ ]:
plt.imshow(test_vol[0].astype(np.float32))

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
tf.config.list_physical_devices('GPU')

from tensorflow.python.client import device_lib

device_lib.list_local_devices()

tf.test.is_built_with_cuda()

tf.debugging.set_log_device_placement(True)

In [ ]:

loss_history = model.fit(x = train_vol,
                       y = train_seg,
                         batch_size = 16,
                  epochs = 300,
                  validation_data =(test_vol,test_seg) ,
                  callbacks=callbacks_list)

model.save('model.h5')

In [ ]:
model.save('attention_model.h5')

In [ ]:
plt.plot(loss_history.history['loss'], '-', label = 'Loss')
plt.plot(loss_history.history['val_loss'], '-', label = 'Validation Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Dice Loss')
plt.show()


In [ ]:
pred_candidates = np.random.randint(1,validation_vol.shape[0],10)
preds = model.predict(validation_vol)

In [ ]:
test_vol.shape[0]

In [ ]:
probs = model.predict(test_vol)

In [ ]:
pred_candidates

In [ ]:
preds.shape

In [ ]:
plt.imshow(preds[1])

In [ ]:


plt.figure(figsize=(20,10))

for i in range(0,9,4):
    plt.subplot(4,4,i+1)
    
    plt.imshow(validation_vol[pred_candidates[i]].astype(np.float32))
    plt.xlabel("Base Image")
    plt.axis('off')
    
    
    plt.subplot(4,4,i+2)
    plt.imshow(np.squeeze(validation_seg[pred_candidates[i]]).astype(np.float32))
    plt.xlabel("Mask")
    plt.axis('off')
    
    plt.subplot(4,4,i+3)
    plt.imshow(np.squeeze(preds[pred_candidates[i]]).astype(np.float32))
    plt.xlabel("Pridiction")
    plt.axis('off')
    
    plt.subplot(4,4,i+4)
    plt.imshow(validation_vol[pred_candidates[i]]+np.squeeze(preds[pred_candidates[i]]*0.2).astype(np.float32))
    plt.xlabel("Pridiction")
    plt.axis('off')
    
    plt.tight_layout()
    

In [ ]:
from tensorflow.keras.models import load_model
from numpy import load
from numpy import vstack
import matplotlib.pyplot as plt
from matplotlib import pyplot
from numpy.random import randint
import tensorflow.keras as tf
# load and prepare training images
def load_real_samples(filename):
    # load compressed arrays
    data = load(filename)
    # unpack arrays
    X1, X2 = data['arr_0'], data['arr_1']
    # scale from [0,255] to [-1,1]
    X1 = ((X1) / 255.0)
    X2 = ((X2) / 255.0)
    return [X1, X2]
ssi = []
psi = []
iou = []
# plot source, generated and target images
def plot_images(src_img, gen_img, tar_img):
    images = vstack((src_img, gen_img, tar_img))
    # scale from [-1,1] to [0,1]
    images = (images + 1) / 2.0
    titles = ['Source', 'Generated', 'Expected']
    # plot images row by row
    fig = plt.figure(figsize = [8,8])
    for i in range(len(images)):
        # define subplot
        ax = pyplot.subplot(1, 3, 1 + i)
        #plt.figure(figsize = [5,5])
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        ax.imshow(np.squeeze(images[i]).astype(np.float32), cmap = 'gray')
        # show title
        ax.set_title(titles[i])
    ss = str(float(ssim1))
    ps = str(float(psnr1))
    io = str(float(iou1))
    
    a =  'SSIM:' ,ss[:5], 'PSNR:' ,  ps[:5], 'IOU=', io[:5]
    print(a)
    
    ssi.append(ss)
    psi.append(ps)
    iou.append(io)
    fig.suptitle(a)
    fig.tight_layout()
    fig.subplots_adjust(top= 1.5)
    #plt.savefig(name, bbox_inches = 'tight', pad_inches = 0)
    
    pyplot.show()

In [ ]:
import tensorflow as tf 
#tf.random.Generator = None  # Patch for a bug
from tensorflow_addons.layers import InstanceNormalization


# for testing images

In [ ]:
[X2, X1] = load_real_samples('../input/unetdata/testing_256.npz')

In [ ]:
[X2, X1] = load_real_samples('../input/unetdata/testing_256.npz')
#print('Loaded', X1.shape, X2.shape)
# load model


import numpy as np 
XD = np.arange(0,65)
for i in range(20):
    print('for image number ', i )
    name = 'mname'+ '_'+str(i) +'_' + '.png'

    ix = XD[i]
    ix = np.array([ix])
    tar_image, src_image   = X1[[ix]], X2[[ix]]
    # generate image from source
    gen_image = model.predict(src_image)
    # plot all three images
    images = vstack((src_image, gen_image, tar_image))
    images = (images + 1) / 2.0
    df = images[1]

    print('RAW image: ', 'Min: %.3f, Max: %.3f' % (df.min(), df.max()))
    df1 = ((df) / df.max())*255
    print('After rescaling: Min: %.3f, Max: %.3f' % (df1.min(), df1.max()))
    dfn = images[2]

    print('RAW image: ', 'Min: %.3f, Max: %.3f' % (dfn.min(), dfn.max()))
    dfn1 = ((dfn) / dfn.max())*255
    print('After rescaling: Min: %.3f, Max: %.3f' % (dfn1.min(), dfn1.max()))

    a =tf.convert_to_tensor(df1, tf.uint8)
    b = tf.convert_to_tensor(dfn1, tf.uint8)
    ssim1 = tf.image.ssim(a,b, max_val=255, filter_size=11,filter_sigma=1.5, k1=0.01, k2=0.03) 
    psnr1 = tf.image.psnr(a, b, max_val=255)
    print(float(ssim1))
    print(float(psnr1))
    im1 = tf.image.convert_image_dtype(a, tf.float32)
    im2 = tf.image.convert_image_dtype(b, tf.float32)
    ssim2 = tf.image.ssim(im1,im2, max_val=1, filter_size=11,
                          filter_sigma=1.5, k1=0.01, k2=0.03)
    # IOU calculation 
    a1 = a[:,:,0]
    img = a1/np.max(a1)
    imgn = np.where(img<0.9,0, 1)

    #plt.imshow(imgn, cmap = 'gray')

    b1 = b[:,:,0]
    imgb = b1/np.max(b1)
    imgb = np.where(imgb<0.9,0, 1)
    #plt.imshow(imgb, cmap = 'gray')

    m = tf.keras.metrics.MeanIoU(num_classes=2)
    m.update_state(imgn, imgb)
    iou1 = m.result().numpy()
    plot_images(src_image, gen_image, tar_image)

In [ ]:
iou1 = []
ssim1 = []
PSNR = []
for i in range(len(iou)):
    iou1.append(float(iou[i]))
    ssim1.append(float(ssi[i]))
    PSNR.append(float(psi[i]))
    
    

In [ ]:
import seaborn as sns
sns.set()
plt.plot(iou1)
plt.ylabel('IOU')
plt.xlabel('Image id')
plt.show()
plt.plot(ssim1)
plt.ylabel('SSIM')
plt.xlabel('Image id')
plt.show()
plt.plot(PSNR)
plt.ylabel('PSNR')
plt.xlabel('Image id')
plt.show()

In [ ]:
import numpy as np 
print('mean iou: ', np.mean(iou1))
print('mean SSIM: ', np.mean(ssim1))
print('mean PSNR: ', np.mean(PSNR))
print('mean iou: ', np.std(iou1))
print('mean SSIM: ', np.std(ssim1))
print('mean PSNR: ', np.std(PSNR))

In [ ]:
df = (pd.DataFrame([iou1, ssim1, PSNR])).transpose()
df.columns = ['iou', 'ssim', 'PSNR']
df.to_csv('valid.csv')

# for training dataset 

In [ ]:
from tensorflow.keras.models import load_model
from numpy import load
from numpy import vstack
import matplotlib.pyplot as plt
from matplotlib import pyplot
from numpy.random import randint
import tensorflow.keras as tf
# load and prepare training images
def load_real_samples(filename):
    # load compressed arrays
    data = load(filename)
    # unpack arrays
    X1, X2 = data['arr_0'], data['arr_1']
    # scale from [0,255] to [-1,1]
    X1 = ((X1) / 255.0)
    X2 = ((X2) / 255.0)
    return [X1, X2]
ssi = []
psi = []
iou = []
# plot source, generated and target images
def plot_images(src_img, gen_img, tar_img):
    images = vstack((src_img, gen_img, tar_img))
    # scale from [-1,1] to [0,1]
    images = (images + 1) / 2.0
    titles = ['Source', 'Generated', 'Expected']
    # plot images row by row
    #fig = plt.figure(figsize = [8,8])
    #for i in range(len(images)):
        # define subplot
        #ax = pyplot.subplot(1, 3, 1 + i)
        #plt.figure(figsize = [5,5])
        # turn off axis
        #pyplot.axis('off')
        # plot raw pixel data
        #ax.imshow(np.squeeze(images[i]).astype(np.float32), cmap = 'gray')
        # show title
        #ax.set_title(titles[i])
    ss = str(float(ssim1))
    ps = str(float(psnr1))
    io = str(float(iou1))
    
    a =  'SSIM:' ,ss[:5], 'PSNR:' ,  ps[:5], 'IOU=', io[:5]
    print(a)
    
    ssi.append(ss)
    psi.append(ps)
    iou.append(io)
    #fig.suptitle(a)
    #fig.tight_layout()
    #fig.subplots_adjust(top= 1.5)
    #plt.savefig(name, bbox_inches = 'tight', pad_inches = 0)
    
    pyplot.show()
    
import tensorflow as tf 
#tf.random.Generator = None  # Patch for a bug
from tensorflow_addons.layers import InstanceNormalization

In [ ]:
import tensorflow as tf 
#tf.random.Generator = None  # Patch for a bug
from tensorflow_addons.layers import InstanceNormalization



In [ ]:
[X2, X1] = load_real_samples('../input/unetdata/training_256.npz')
#print('Loaded', X1.shape, X2.shape)
# load model


import numpy as np 
XD = np.arange(0,555-20)
for i in range(555-20):
    print('for image number ', i )
    name = 'mname'+ '_'+str(i) +'_' + '.png'

    ix = XD[i]
    ix = np.array([ix])
    tar_image, src_image   = X1[[ix]], X2[[ix]]
    # generate image from source
    gen_image = model.predict(src_image)
    # plot all three images
    images = vstack((src_image, gen_image, tar_image))
    images = (images + 1) / 2.0
    df = images[1]

    print('RAW image: ', 'Min: %.3f, Max: %.3f' % (df.min(), df.max()))
    df1 = ((df) / df.max())*255
    print('After rescaling: Min: %.3f, Max: %.3f' % (df1.min(), df1.max()))
    dfn = images[2]

    print('RAW image: ', 'Min: %.3f, Max: %.3f' % (dfn.min(), dfn.max()))
    dfn1 = ((dfn) / dfn.max())*255
    print('After rescaling: Min: %.3f, Max: %.3f' % (dfn1.min(), dfn1.max()))

    a =tf.convert_to_tensor(df1, tf.uint8)
    b = tf.convert_to_tensor(dfn1, tf.uint8)
    ssim1 = tf.image.ssim(a,b, max_val=255, filter_size=11,filter_sigma=1.5, k1=0.01, k2=0.03) 
    psnr1 = tf.image.psnr(a, b, max_val=255)
    print(float(ssim1))
    print(float(psnr1))
    im1 = tf.image.convert_image_dtype(a, tf.float32)
    im2 = tf.image.convert_image_dtype(b, tf.float32)
    ssim2 = tf.image.ssim(im1,im2, max_val=1, filter_size=11,
                          filter_sigma=1.5, k1=0.01, k2=0.03)
    # IOU calculation 
    a1 = a[:,:,0]
    img = a1/np.max(a1)
    imgn = np.where(img<0.9,0, 1)

    #plt.imshow(imgn, cmap = 'gray')

    b1 = b[:,:,0]
    imgb = b1/np.max(b1)
    imgb = np.where(imgb<0.9,0, 1)
    #plt.imshow(imgb, cmap = 'gray')

    m = tf.keras.metrics.MeanIoU(num_classes=2)
    m.update_state(imgn, imgb)
    iou1 = m.result().numpy()
    plot_images(src_image, gen_image, tar_image)

In [ ]:
iou1 = []
ssim1 = []
PSNR = []
for i in range(len(iou)):
    iou1.append(float(iou[i]))
    ssim1.append(float(ssi[i]))
    PSNR.append(float(psi[i]))

In [ ]:
import seaborn as sns
sns.set()
plt.plot(iou1)
plt.ylabel('IOU')
plt.xlabel('Image id')
plt.show()
plt.plot(ssim1)
plt.ylabel('SSIM')
plt.xlabel('Image id')
plt.show()
plt.plot(PSNR)
plt.ylabel('PSNR')
plt.xlabel('Image id')
plt.show()

In [ ]:
import numpy as np 
print('mean iou: ', np.mean(iou1))
print('mean SSIM: ', np.mean(ssim1))
print('mean PSNR: ', np.mean(PSNR))
print('mean iou: ', np.std(iou1))
print('mean SSIM: ', np.std(ssim1))
print('mean PSNR: ', np.std(PSNR))

In [ ]:
df = (pd.DataFrame([iou1, ssim1, PSNR])).transpose()
df.columns = ['iou', 'ssim', 'PSNR']
df.to_csv('train.csv')

# final testing images

In [ ]:
from tensorflow.keras.models import load_model
from numpy import load
from numpy import vstack
import matplotlib.pyplot as plt
from matplotlib import pyplot
from numpy.random import randint
import tensorflow.keras as tf
# load and prepare training images
def load_real_samples(filename):
    # load compressed arrays
    data = load(filename)
    # unpack arrays
    X1, X2 = data['arr_0'], data['arr_1']
    # scale from [0,255] to [-1,1]
    X1 = ((X1) / 255.0)
    X2 = ((X2) / 255.0)
    return [X1, X2]
ssi = []
psi = []
iou = []
# plot source, generated and target images
def plot_images(src_img, gen_img, tar_img):
    images = vstack((src_img, gen_img, tar_img))
    # scale from [-1,1] to [0,1]
    images = (images + 1) / 2.0
    titles = ['Source', 'Generated', 'Expected']
    # plot images row by row
    #fig = plt.figure(figsize = [8,8])
    #for i in range(len(images)):
        # define subplot
        #ax = pyplot.subplot(1, 3, 1 + i)
        #plt.figure(figsize = [5,5])
        # turn off axis
        #pyplot.axis('off')
        # plot raw pixel data
        #ax.imshow(np.squeeze(images[i]).astype(np.float32), cmap = 'gray')
        # show title
        #ax.set_title(titles[i])
    ss = str(float(ssim1))
    ps = str(float(psnr1))
    io = str(float(iou1))
    
    a =  'SSIM:' ,ss[:5], 'PSNR:' ,  ps[:5], 'IOU=', io[:5]
    print(a)
    
    ssi.append(ss)
    psi.append(ps)
    iou.append(io)
    #fig.suptitle(a)
    #fig.tight_layout()
    #fig.subplots_adjust(top= 1.5)
    #plt.savefig(name, bbox_inches = 'tight', pad_inches = 0)
    
    pyplot.show()
    
import tensorflow as tf 
#tf.random.Generator = None  # Patch for a bug
from tensorflow_addons.layers import InstanceNormalization

In [ ]:
[X2, X1] = load_real_samples('../input/finaldata/finaltesting.npz')
#print('Loaded', X1.shape, X2.shape)
# load model

#print('Loaded', X1.shape, X2.shape)
# load model


import numpy as np 
XD = np.arange(0,180)
for i in range(180):
    print('for image number ', i )
    name = 'mname'+ '_'+str(i) +'_' + '.png'

    ix = XD[i]
    ix = np.array([ix])
    tar_image, src_image   = X1[[ix]], X2[[ix]]
    # generate image from source
    gen_image = model.predict(src_image)
    # plot all three images
    images = vstack((src_image, gen_image, tar_image))
    images = (images + 1) / 2.0
    df = images[1]

    print('RAW image: ', 'Min: %.3f, Max: %.3f' % (df.min(), df.max()))
    df1 = ((df) / df.max())*255
    print('After rescaling: Min: %.3f, Max: %.3f' % (df1.min(), df1.max()))
    dfn = images[2]

    print('RAW image: ', 'Min: %.3f, Max: %.3f' % (dfn.min(), dfn.max()))
    dfn1 = ((dfn) / dfn.max())*255
    print('After rescaling: Min: %.3f, Max: %.3f' % (dfn1.min(), dfn1.max()))

    a =tf.convert_to_tensor(df1, tf.uint8)
    b = tf.convert_to_tensor(dfn1, tf.uint8)
    ssim1 = tf.image.ssim(a,b, max_val=255, filter_size=11,filter_sigma=1.5, k1=0.01, k2=0.03) 
    psnr1 = tf.image.psnr(a, b, max_val=255)
    print(float(ssim1))
    print(float(psnr1))
    im1 = tf.image.convert_image_dtype(a, tf.float32)
    im2 = tf.image.convert_image_dtype(b, tf.float32)
    ssim2 = tf.image.ssim(im1,im2, max_val=1, filter_size=11,
                          filter_sigma=1.5, k1=0.01, k2=0.03)
    # IOU calculation 
    a1 = a[:,:,0]
    img = a1/np.max(a1)
    imgn = np.where(img<0.9,0, 1)

    #plt.imshow(imgn, cmap = 'gray')

    b1 = b[:,:,0]
    imgb = b1/np.max(b1)
    imgb = np.where(imgb<0.9,0, 1)
    #plt.imshow(imgb, cmap = 'gray')

    m = tf.keras.metrics.MeanIoU(num_classes=2)
    m.update_state(imgn, imgb)
    iou1 = m.result().numpy()
    plot_images(src_image, gen_image, tar_image)

In [ ]:
iou1 = []
ssim1 = []
PSNR = []
for i in range(len(iou)):
    iou1.append(float(iou[i]))
    ssim1.append(float(ssi[i]))
    PSNR.append(float(psi[i]))

In [ ]:
import numpy as np 
print('mean iou: ', np.mean(iou1))
print('mean SSIM: ', np.mean(ssim1))
print('mean PSNR: ', np.mean(PSNR))
print('mean iou: ', np.std(iou1))
print('mean SSIM: ', np.std(ssim1))
print('mean PSNR: ', np.std(PSNR))

In [ ]:
import seaborn as sns
sns.set()
plt.plot(iou1)
plt.ylabel('IOU')
plt.xlabel('Image id')
plt.show()
plt.plot(ssim1)
plt.ylabel('SSIM')
plt.xlabel('Image id')
plt.show()
plt.plot(PSNR)
plt.ylabel('PSNR')
plt.xlabel('Image id')
plt.show()

In [ ]:
import numpy as np 
print('mean iou: ', np.mean(iou1))
print('mean SSIM: ', np.mean(ssim1))
print('mean PSNR: ', np.mean(PSNR))
print('mean iou: ', np.std(iou1))
print('mean SSIM: ', np.std(ssim1))
print('mean PSNR: ', np.std(PSNR))

In [ ]:
import seaborn as sns
sns.set()
plt.plot(iou1)
plt.ylabel('IOU')
plt.xlabel('Image id')
plt.show()
plt.plot(ssim1)
plt.ylabel('SSIM')
plt.xlabel('Image id')
plt.show()
plt.plot(PSNR)
plt.ylabel('PSNR')
plt.xlabel('Image id')
plt.show()

In [ ]:
df = (pd.DataFrame([iou1, ssim1, PSNR])).transpose()
df.columns = ['iou', 'ssim', 'PSNR']
df.to_csv('testfinal.csv')